# LINEAR_PROG

## Overview
The `linear_prog` function solves linear programming (LP) problems using the `scipy.optimize.linprog` function. This is useful for business users in Excel who need to optimize resource allocation, minimize costs, or maximize profits subject to linear constraints. The function accepts the objective coefficients, constraint matrices, and bounds as arguments, and returns the optimal solution and value, or an error message (as a string) if the problem is infeasible or input is invalid.

## Usage
To use the `LINEAR_PROG` function in Excel, enter it as a formula in a cell, specifying the required arguments:

```excel
=LINEAR_PROG(c, A_ub, b_ub, A_eq, b_eq, bounds, method)
```

- `c`: 2D list (row or column vector) of objective coefficients (to minimize c^T x), e.g., `[[3, 5]]`
- `A_ub`: 2D list for inequality constraints (`A_ub x <= b_ub`)
- `b_ub`: 2D list (row or column vector) for inequality constraint bounds, e.g., `[[-8], [-8]]`
- `A_eq`: 2D list for equality constraints (`A_eq x == b_eq`)
- `b_eq`: 2D list (row or column vector) for equality constraint bounds
- `bounds`: 2D list of variable bounds `[[min1, max1], [min2, max2], ...]`
- `method`: (optional) string, LP algorithm to use (e.g., `'highs'`, `'interior-point'`, `'revised simplex'`)

## Arguments
| Argument | Type | Required | Description |
|----------|------|----------|-------------|
| c | 2D list or scalar | Yes | Coefficients for the linear objective function to be minimized (as a row or column vector, or scalar) |
| A_ub | 2D list or scalar | No | Coefficient matrix for inequality constraints (`A_ub x <= b_ub`) |
| b_ub | 2D list or scalar | No | Right-hand side vector for inequality constraints (as a row or column vector, or scalar) |
| A_eq | 2D list or scalar | No | Coefficient matrix for equality constraints (`A_eq x == b_eq`) |
| b_eq | 2D list or scalar | No | Right-hand side vector for equality constraints (as a row or column vector, or scalar) |
| bounds | 2D list or scalar | No | Bounds for variables as `[[min, max], ...]` or a single `[min, max]` pair |
| method | string | No | LP algorithm to use: `'simplex'`, `'interior-point'`, `'revised simplex'`, `'highs-ipm'`, `'highs-ds'`, `'highs'` (default: `'highs'`) |

## Returns
| Return Value | Type | Description |
|-------------|------|-------------|
| result | 2D list | `[[x1, x2, ..., xn], fun]`: optimal variable values and optimal value |
| error | string | Error message if input is invalid or problem is infeasible (returned as a string) |

## Examples

### Resource Allocation (Minimize Cost)
A company wants to minimize cost: Minimize `3x + 5y`, subject to:
- `x + 2y >= 8`
- `2x + y >= 8`
- `x >= 0`, `y >= 0`

**Excel Setup:**
- Cell A1: `[[3, 5]]` (Objective coefficients as a row vector)
- Cell B1: `[[-1, -2], [-2, -1]]` (`A_ub` for `>=` constraints as `<=`)
- Cell C1: `[[-8], [-8]]` (`b_ub` as a column vector)
- Cell D1: (leave blank)
- Cell E1: (leave blank)
- Cell F1: `[[0, None], [0, None]]` (bounds)

**Formula in Excel:**
`=LINEAR_PROG(A1, B1, C1, D1, E1, F1)`

**Expected Outcome:**
Returns a 2D list: `[[x, y], minimum cost]`

### Diet Problem (Maximize Protein)
Maximize `x + y` subject to:
- `x + 2y <= 10`
- `x >= 0`, `y >= 0`

**Excel Setup:**
- Cell A2: `[[-1, -1]]` (Negate for minimization, as a row vector)
- Cell B2: `[[1, 2]]` (`A_ub`)
- Cell C2: `[[10]]` (`b_ub` as a column vector)
- Cell D2: (leave blank)
- Cell E2: (leave blank)
- Cell F2: `[[0, None], [0, None]]` (bounds)

**Formula in Excel:**
`=LINEAR_PROG(A2, B2, C2, D2, E2, F2)`

**Expected Outcome:**
Returns a 2D list: `[[x, y], maximum protein]`

### Infeasible Problem
If the constraints are contradictory, the function returns an error message (as a string).

In [1]:
from scipy.optimize import linprog
import numpy as np

def linear_prog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method=None):
    """
    Solves a linear programming problem using scipy.optimize.linprog.
    Args:
        c (2D list or scalar): Coefficients for the linear objective function to be minimized (as column vector or scalar).
        A_ub (2D list or scalar, optional): 2D list for inequality constraints (A_ub x <= b_ub).
        b_ub (2D list or scalar, optional): 2D list for inequality constraint bounds (each row is a single value).
        A_eq (2D list or scalar, optional): 2D list for equality constraints (A_eq x == b_eq).
        b_eq (2D list or scalar, optional): 2D list for equality constraint bounds (each row is a single value).
        bounds (2D list or scalar, optional): 2D list of variable bounds [[min, max], ...].
        method (str, optional): LP algorithm to use (e.g., 'highs').
    Returns:
        list: [[x1, x2, ...], fun] (optimal variable values and optimal value), or error string
    """
    try:
        def to_1d(arr, name):
            if arr is None:
                return None
            if isinstance(arr, (int, float)):
                return np.array([arr])
            arr = np.array(arr)
            if arr.ndim == 2 and (arr.shape[1] == 1 or arr.shape[0] == 1):
                return arr.flatten()
            return None
        def to_2d(arr, name):
            if arr is None:
                return None
            if isinstance(arr, (int, float)):
                return np.array([[arr]])
            arr = np.array(arr)
            if arr.ndim == 2:
                return arr
            return None
        c_vec = to_1d(c, 'c')
        if c_vec is None or not np.issubdtype(c_vec.dtype, np.number):
            return "c must be a 2D list (column or row vector) or scalar of numbers."
        n_vars = c_vec.size
        if A_ub is not None:
            A_ub_mat = to_2d(A_ub, 'A_ub')
            if A_ub_mat is None or A_ub_mat.shape[1] != n_vars:
                return "A_ub must be a 2D list with each row of length equal to c, or a scalar."
        else:
            A_ub_mat = None
        if b_ub is not None:
            b_ub_vec = to_1d(b_ub, 'b_ub')
            if b_ub_vec is None:
                return "b_ub must be a 2D list (column or row vector) or scalar."
        else:
            b_ub_vec = None
        if A_eq is not None:
            A_eq_mat = to_2d(A_eq, 'A_eq')
            if A_eq_mat is None or A_eq_mat.shape[1] != n_vars:
                return "A_eq must be a 2D list with each row of length equal to c, or a scalar."
        else:
            A_eq_mat = None
        if b_eq is not None:
            b_eq_vec = to_1d(b_eq, 'b_eq')
            if b_eq_vec is None:
                return "b_eq must be a 2D list (column or row vector) or scalar."
        else:
            b_eq_vec = None
        if bounds is not None:
            bounds_mat = to_2d(bounds, 'bounds')
            if bounds_mat is None or bounds_mat.shape[1] != 2:
                return "bounds must be a 2D list of [min, max] pairs or a scalar."
            bounds_list = [tuple(row) for row in bounds_mat]
        else:
            bounds_list = None
        if method is not None and not isinstance(method, str):
            return "method must be a string or None."
        lp_method = method if method is not None else 'highs'
        result = linprog(
            c_vec,
            A_ub=A_ub_mat,
            b_ub=b_ub_vec,
            A_eq=A_eq_mat,
            b_eq=b_eq_vec,
            bounds=bounds_list,
            method=lp_method
        )
        if not result.success:
            return f"Linear programming failed: {result.message}"
        return [[*list(result.x), float(result.fun)]]
    except Exception as e:
        return f"Error during linear programming: {str(e)}"

In [2]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_resource_allocation_min_cost():
    c = [[3, 5]]
    A_ub = [[-1, -2], [-2, -1]]
    b_ub = [[-8], [-8]]
    bounds = [[0, None], [0, None]]
    result = linear_prog(c, A_ub, b_ub, None, None, bounds)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 3
    assert all(isinstance(x, (float, int)) for x in result[0])

def test_diet_problem_max_protein():
    c = [[-1, -1]]
    A_ub = [[1, 2]]
    b_ub = [[10]]
    bounds = [[0, None], [0, None]]
    result = linear_prog(c, A_ub, b_ub, None, None, bounds)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 3
    assert all(isinstance(x, (float, int)) for x in result[0])

def test_infeasible_problem():
    c = [[1]]
    A_ub = [[-1], [1]]
    b_ub = [[-1], [0]]
    bounds = [[0, 1]]
    result = linear_prog(c, A_ub, b_ub, None, None, bounds)
    assert isinstance(result, str)
    assert ("error" in result.lower() or "fail" in result.lower() or "must be" in result.lower())

def test_invalid_input():
    c = "not_a_list"
    result = linear_prog(c)
    assert isinstance(result, str)
    assert ("error" in result.lower() or "fail" in result.lower() or "must be" in result.lower())

ipytest.run()

....                                                                                         [100%]...                                                                                         [100%]
4 passed in 0.03s

4 passed in 0.03s


<ExitCode.OK: 0>

In [3]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [[3, 5]],
        [[-1, -2], [-2, -1]],
        [[-8], [-8]],
        [[0, 0]],  # A_eq as a 2D list with correct shape but no effect
        [[0]],     # b_eq as a 2D list with correct shape but no effect
        [[0, None], [0, None]],
        "highs"
    ],
    [
        [[-1, -1]],
        [[1, 2]],
        [[10]],
        [[0, 0]],  # A_eq as a 2D list with correct shape but no effect
        [[0]],     # b_eq as a 2D list with correct shape but no effect
        [[0, None], [0, None]],
        "highs"
    ]
]

demo = gr.Interface(
    fn=linear_prog,
    inputs=[
        gr.Dataframe(headers=["c1", "c2"], label="Objective Coefficients (c)", row_count=1, col_count=2, type="array", value=[[3, 5]]),
        gr.Dataframe(headers=["x1", "x2"], label="A_ub (Inequality Coefficients)", row_count=2, col_count=2, type="array", value=[[-1, -2], [-2, -1]]),
        gr.Dataframe(headers=["b_ub"], label="b_ub (Inequality Bounds)", row_count=2, col_count=1, type="array", value=[[-8], [-8]]),
        gr.Dataframe(headers=["x1", "x2"], label="A_eq (Equality Coefficients)", row_count=1, col_count=2, type="array", value=[[0, 0]]),
        gr.Dataframe(headers=["b_eq"], label="b_eq (Equality Bounds)", row_count=1, col_count=1, type="array", value=[[0]]),
        gr.Dataframe(headers=["min", "max"], label="Bounds", row_count=2, col_count=2, type="array", value=[[0, None], [0, None]]),
        gr.Textbox(label="Method (optional)", value="highs")
    ],
    outputs=gr.Dataframe(headers=["x1", "x2", "Optimal Value"], label="Result"),
    examples=examples,
    description="Solve a linear programming problem using scipy.optimize.linprog. Set up your coefficients and constraints as shown in the examples.",
    flagging_mode="never",
)
demo.launch()

c:\Users\brent\Code\python-apps\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
